# Лабораторная работа №1 (Проведение исследований с алгоритмом KNN)

## Выбор начальных условий

### Rice MSC Dataset для задачи классификации
- **Описание**: датасет содержит данные о различных сортах риса, с такими характеристиками, как форма, цвет и др.
- **Возможная задача**: Классификация различных сортов риса по заданным характеристикам.
- **Обоснование**: Это практическая задача, которая может быть полезна в агрономии и пищевой промышленности, где важно правильно идентифицировать сорта.

### Gold Price Regression для задачи регрессии
- **Описание**: Датасет содержит исторические данные о ценах на золото, с такими признаками, как дата, стоимость, экономические индикаторы и т. д.
- **Возможная задача**: Прогнозирование цен на золото на основе исторических данных и других существующих метрик.
- **Обоснование**: Исследование изменений цен на золото актуально для инвестиционной стратегии и экономического анализа.

### Выбор метрик качества

#### Для задачи классификации:
- **Accuracy**: Общее понимание точности модели в предсказании сортов.
- **F1-score**: Уместен, если классы в датасете несбалансированы, так как учитывает и Precision, и Recall.

#### Для задачи регрессии:
- **Mean Absolute Error (MAE**): Измеряет среднюю ошибку в прогнозировании цен на золото.
- **R-squared**: Оценивает, как хорошо модель объясняет изменения цен.

## Создание бейзлайна и оценка качества

### Обучение моделей из sklearn для выбранных наборов данных

#### Импорт библиотек

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer
from scipy.spatial import distance
import numpy as np
from collections import Counter

#### Подготовка данных

##### Классификация (Rice MSC Dataset)

**Загрузка данных:**

In [2]:
rice_data = pd.read_csv('Rice_MSC_Dataset.csv')

X_rice = rice_data.iloc[:, :-1]  # Признаки
y_rice = rice_data.iloc[:, -1]   # Метки классов

**Разделение на тренировочный и тестовый наборы:**

In [3]:
X_train_rice, X_test_rice, y_train_rice, y_test_rice = train_test_split(X_rice, y_rice, test_size=0.2, random_state=42)

**Замена пропущенных значений**

In [4]:
imputer = SimpleImputer(strategy='mean')
X_train_rice = pd.DataFrame(imputer.fit_transform(X_train_rice))
X_test_rice = pd.DataFrame(imputer.transform(X_test_rice))

##### Регрессия (Gold Price Regression)

**Загрузка данных:**

In [5]:
gold_data = pd.read_csv('financial_regression.csv')

X_gold = gold_data.iloc[:, 1:-1]  # Признаки
y_gold = gold_data.iloc[:, -1]   # Целевая переменная

**Разделение на тренировочный и тестовый наборы:**

In [6]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)

**Замена пропущенных значений:**

In [7]:
imputer = SimpleImputer(strategy='mean')
X_train_gold = imputer.fit_transform(X_train_gold)
X_test_gold = imputer.transform(X_test_gold)

**Удаление строк с пропущенными значениями:**

In [8]:
mask = ~np.isnan(y_train_gold)
X_train_gold = X_train_gold[mask]
y_train_gold = y_train_gold[mask]

mask_test = ~np.isnan(y_test_gold)
X_test_gold = X_test_gold[mask_test]
y_test_gold = y_test_gold[mask_test]

#### Обучение моделей

##### Классификация

In [9]:
knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train_rice, y_train_rice)

KNeighborsClassifier()

##### Регрессия

In [10]:
knn_reg = KNeighborsRegressor(n_neighbors=5)
knn_reg.fit(X_train_gold, y_train_gold)

KNeighborsRegressor()

### Оценка качества моделей по выбранным метрикам

##### Классификация

In [11]:
y_pred_rice_origin = knn_clf.predict(X_test_rice)
accuracy_origin = accuracy_score(y_test_rice, y_pred_rice_origin)
f1_origin = f1_score(y_test_rice, y_pred_rice_origin, average='weighted')
print(f"Accuracy: {accuracy_origin}")
print(f"F1 Score: {f1_origin}")

Accuracy: 0.9230666666666667
F1 Score: 0.922516443102621


##### Регрессия

In [12]:
y_pred_gold_origin = knn_reg.predict(X_test_gold)
mae_origin = mean_absolute_error(y_test_gold, y_pred_gold_origin)
r2_origin = r2_score(y_test_gold, y_pred_gold_origin)
print(f"Mean Absolute Error: {mae_origin}")
print(f"R-squared: {r2_origin}")

Mean Absolute Error: 3605268.701468624
R-squared: 0.26023132479378674


## Улучшение бейзлайна

### Гипотезы

1. Улучшение предобработки данных:
   - Масштабирование признаков: проверить необходимость нормализации или стандартизации.

2. Формирование новых признаков:
   - Создание новых признаков на основе комбинации существующих, с учетом специфики данных.

3. Подбор гиперпараметров:
   - Использование Grid Search с кросс-валидацией для оптимизации модели.

### Проверка гипотез

#### 1. Препроцессинг данных

##### Классификация

Масштабирование признаков:

In [13]:
scaler_rice = StandardScaler()
X_train_rice_with_scaler = scaler_rice.fit_transform(X_train_rice)
X_test_rice_with_scaler = scaler_rice.transform(X_test_rice)

Обучение модели:

In [14]:
knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train_rice_with_scaler, y_train_rice)

KNeighborsClassifier()

Оценка качества модели:

In [15]:
y_pred_rice = knn_clf.predict(X_test_rice_with_scaler)
accuracy = accuracy_score(y_test_rice, y_pred_rice)
f1 = f1_score(y_test_rice, y_pred_rice, average='weighted')
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Accuracy: 0.9976
F1 Score: 0.9976013022879141


##### Регрессия

Масштабирование признаков:

In [16]:
scaler_gold = StandardScaler()
X_train_gold_with_scaler = scaler_gold.fit_transform(X_train_gold)
X_test_gold_with_scaler = scaler_gold.transform(X_test_gold)

Обучение модели:

In [17]:
knn_reg = KNeighborsRegressor(n_neighbors=5)
knn_reg.fit(X_train_gold_with_scaler, y_train_gold)

KNeighborsRegressor()

Оценка качества модели:

In [18]:
y_pred_gold = knn_reg.predict(X_test_gold_with_scaler)
mae = mean_absolute_error(y_test_gold, y_pred_gold)
r2 = r2_score(y_test_gold, y_pred_gold)
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")

Mean Absolute Error: 2456822.1217623497
R-squared: 0.5848677706844285


#### 2. Формирование новых признаков

##### Классификация

Добавления нового признака:

In [19]:
X_rice_with_new_future = X_rice.copy()

In [20]:
X_rice_with_new_future["new_future"] = X_rice_with_new_future["AREA"] * X_rice_with_new_future["PERIMETER"]

Разделение на тренировочный и тестовый наборы:

In [21]:
X_train_rice, X_test_rice, y_train_rice, y_test_rice = train_test_split(X_rice_with_new_future, y_rice, test_size=0.2, random_state=42)

Замена пропущенных значений:

In [22]:
imputer = SimpleImputer(strategy='mean')
X_train_rice = pd.DataFrame(imputer.fit_transform(X_train_rice))
X_test_rice = pd.DataFrame(imputer.transform(X_test_rice))

Масштабирование признаков:

In [23]:
scaler_rice = StandardScaler()
X_train_rice = scaler_rice.fit_transform(X_train_rice)
X_test_rice = scaler_rice.transform(X_test_rice)

Обучение модели:

In [24]:
knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train_rice, y_train_rice)

KNeighborsClassifier()

Оценка качества модели:

In [25]:
y_pred_rice = knn_clf.predict(X_test_rice)
accuracy = accuracy_score(y_test_rice, y_pred_rice)
f1 = f1_score(y_test_rice, y_pred_rice, average='weighted')
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Accuracy: 0.9975333333333334
F1 Score: 0.9975346880306323


##### Регрессия

Добавления нового признака:

In [26]:
X_gold_with_new_future = X_gold.copy()

In [27]:
X_gold_with_new_future["new_future"] = X_gold_with_new_future["silver open"] * X_gold_with_new_future["oil open"]

Разделение на тренировочный и тестовый наборы:

In [28]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold_with_new_future, y_gold, test_size=0.2, random_state=42)

Замена пропущенных значений:

In [29]:
imputer = SimpleImputer(strategy='mean')
X_train_gold = imputer.fit_transform(X_train_gold)
X_test_gold = imputer.transform(X_test_gold)

Удаление строк с пропущенными значениями:

In [30]:
mask = ~np.isnan(y_train_gold)
X_train_gold = X_train_gold[mask]
y_train_gold = y_train_gold[mask]

mask_test = ~np.isnan(y_test_gold)
X_test_gold = X_test_gold[mask_test]
y_test_gold = y_test_gold[mask_test]

Масштабирование признаков:

In [31]:
scaler_gold = StandardScaler()
X_train_gold = scaler_gold.fit_transform(X_train_gold)
X_test_gold = scaler_gold.transform(X_test_gold)

Обучение модели:

In [32]:
knn_reg = KNeighborsRegressor(n_neighbors=5)
knn_reg.fit(X_train_gold, y_train_gold)

KNeighborsRegressor()

Оценка качества модели:

In [33]:
y_pred_gold = knn_reg.predict(X_test_gold)
mae = mean_absolute_error(y_test_gold, y_pred_gold)
r2 = r2_score(y_test_gold, y_pred_gold)
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")

Mean Absolute Error: 2458516.087850468
R-squared: 0.5869693562024589


#### 3. Подбор гиперпараметров

##### Классификация

Разделение на тренировочный и тестовый наборы:

In [34]:
X_train_rice, X_test_rice, y_train_rice, y_test_rice = train_test_split(X_rice, y_rice, test_size=0.2, random_state=42)

Замена пропущенных значений:

In [35]:
imputer = SimpleImputer(strategy='mean')
X_train_rice = pd.DataFrame(imputer.fit_transform(X_train_rice))
X_test_rice = pd.DataFrame(imputer.transform(X_test_rice))

Масштабирование признаков:

In [36]:
scaler_rice = StandardScaler()
X_train_rice = scaler_rice.fit_transform(X_train_rice)
X_test_rice = scaler_rice.transform(X_test_rice)

Определение параметров для перебора:

In [37]:
param_grid = {
    'n_neighbors': range(1, 5),
    'weights': ['uniform', 'distance'],
}

Настройка и обучение GridSearchCV:

In [38]:
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)
grid_search.fit(X_train_rice, y_train_rice)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 5),
                         'weights': ['uniform', 'distance']})

Вывод лучших параметров и результата:

In [39]:
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)

Best parameters found:  {'n_neighbors': 3, 'weights': 'uniform'}
Best cross-validation accuracy:  0.9978833333333332


Оценка качества модели:

In [40]:
best_knn = grid_search.best_estimator_
y_pred = best_knn.predict(X_test_rice)
accuracy = accuracy_score(y_test_rice, y_pred)
f1 = f1_score(y_test_rice, y_pred, average='weighted')
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Accuracy: 0.9978666666666667
F1 Score: 0.9978677682961133


##### Регрессия

Разделение на тренировочный и тестовый наборы:

In [41]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)

Замена пропущенных значений:

In [42]:
imputer = SimpleImputer(strategy='mean')
X_train_gold = imputer.fit_transform(X_train_gold)
X_test_gold = imputer.transform(X_test_gold)

Удаление строк с пропущенными значениями:

In [43]:
mask = ~np.isnan(y_train_gold)
X_train_gold = X_train_gold[mask]
y_train_gold = y_train_gold[mask]

mask_test = ~np.isnan(y_test_gold)
X_test_gold = X_test_gold[mask_test]
y_test_gold = y_test_gold[mask_test]

Масштабирование признаков:

In [44]:
scaler_gold = StandardScaler()
X_train_gold = scaler_gold.fit_transform(X_train_gold)
X_test_gold = scaler_gold.transform(X_test_gold)

Определение параметров для перебора:

In [45]:
param_grid = {
    'n_neighbors': range(1, 5),
    'weights': ['uniform', 'distance'],
    'metric': ['manhattan', 'euclidean']
}

Настройка и обучение GridSearchCV:

In [46]:
grid_search = GridSearchCV(KNeighborsRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_gold, y_train_gold)

GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid={'metric': ['manhattan', 'euclidean'],
                         'n_neighbors': range(1, 5),
                         'weights': ['uniform', 'distance']},
             scoring='neg_mean_squared_error')

Вывод лучших параметров и результата:

In [47]:
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)

Best parameters found:  {'metric': 'manhattan', 'n_neighbors': 4, 'weights': 'distance'}
Best cross-validation accuracy:  -14375992395361.484


Оценка модели с лучшими параметрами на тестовом наборе:

In [48]:
best_knn = grid_search.best_estimator_
y_pred = best_knn.predict(X_test_gold)
mae = mean_absolute_error(y_test_gold, y_pred)
r2 = r2_score(y_test_gold, y_pred)
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")

Mean Absolute Error: 2398788.410257056
R-squared: 0.6048118747567148


### Формирование улучшенного бейзлайна

##### Классификация

В улучшенном бейзлайне добавлено масштабирование признаков, а значение гиперпараметра n_neighbors изменено на 3.

Формирование улучшенного бейзлайна:

In [49]:
X_train_rice, X_test_rice, y_train_rice, y_test_rice = train_test_split(X_rice, y_rice, test_size=0.2, random_state=42)

imputer = SimpleImputer(strategy='mean')
X_train_rice = pd.DataFrame(imputer.fit_transform(X_train_rice))
X_test_rice = pd.DataFrame(imputer.transform(X_test_rice))

scaler_rice = StandardScaler()
X_train_rice = scaler_rice.fit_transform(X_train_rice)
X_test_rice = scaler_rice.transform(X_test_rice)
                                   
knn_clf = KNeighborsClassifier(n_neighbors=3)

##### Регрессия

В улучшенном бейзлайне добавлено масштабирование признаков, значение гиперпараметра n_neighbors изменено на 4, значение гиперпараметра weights изменено на "distance", занчение metric -- на "manhattan"

In [50]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)

imputer = SimpleImputer(strategy='mean')
X_train_gold = imputer.fit_transform(X_train_gold)
X_test_gold = imputer.transform(X_test_gold)

mask = ~np.isnan(y_train_gold)
X_train_gold = X_train_gold[mask]
y_train_gold = y_train_gold[mask]
mask_test = ~np.isnan(y_test_gold)
X_test_gold = X_test_gold[mask_test]
y_test_gold = y_test_gold[mask_test]

scaler_gold = StandardScaler()
X_train_gold = scaler_gold.fit_transform(X_train_gold)
X_test_gold = scaler_gold.transform(X_test_gold)

knn_reg = KNeighborsRegressor(n_neighbors=4, weights="distance", metric="manhattan")

### Обучение модели с улучшенным бейзлайном

##### Классификация

Обучение модели:

In [51]:
knn_clf.fit(X_train_rice, y_train_rice)

KNeighborsClassifier(n_neighbors=3)

##### Регрессия

Обучение модели:

In [52]:
knn_reg.fit(X_train_gold, y_train_gold)

KNeighborsRegressor(metric='manhattan', n_neighbors=4, weights='distance')

### Оценка качества модели с улучшенным бейзлайном

##### Классификация

Оценка качества модели:

In [53]:
y_pred_rice_improved = knn_clf.predict(X_test_rice)
accuracy_improved = accuracy_score(y_test_rice, y_pred_rice)
f1_improved = f1_score(y_test_rice, y_pred_rice, average='weighted')
print(f"Accuracy: {accuracy_improved}")
print(f"F1 Score: {f1_improved}")

Accuracy: 0.9975333333333334
F1 Score: 0.9975346880306323


##### Регрессия

Оценка качества модели:

In [54]:
y_pred_gold_improved = knn_reg.predict(X_test_gold)
mae_improved = mean_absolute_error(y_test_gold, y_pred_gold)
r2_improved = r2_score(y_test_gold, y_pred_gold)
print(f"Mean Absolute Error: {mae_improved}")
print(f"R-squared: {r2_improved}")

Mean Absolute Error: 2458516.087850468
R-squared: 0.5869693562024589


### Сравнение результатов первоначального бейзлайна с улучшенным бейзлайном

##### Классификация

Видим, что результаты улучшились:

In [55]:
print(f"Accuracy difference: {accuracy_improved-accuracy_origin}")
print(f"F1 Score difference: {f1_improved-f1_origin}")

Accuracy difference: 0.07446666666666668
F1 Score difference: 0.07501824492801124


##### Регрессия

Видим, что результаты улучшились:

In [56]:
print(f"Mean Absolute Error difference: {mae_origin-mae_improved}")
print(f"R-squared difference: {r2_improved-r2_origin}")

Mean Absolute Error difference: 1146752.6136181564
R-squared difference: 0.3267380314086722


### Выводы

С улучшенным бейзлайном качество модели возрасло, поэтому можно сделать следующие выводы:
1. Масштабирование важно: Методы, такие как K-ближайшие соседи, чувствительны к масштабу признаков. Масштабирование улучшает точность, так как делает все признаки равнозначными.
2. Гиперпараметры имеют значение: Оптимальная настройка гиперпараметров, таких как количество соседей (n_neighbors), значительно повышает качество модели, если сравнивать с настройками по умолчанию.

## Имплементация алгоритма машинного обучения

### Самостоятельная имплементация

##### Классификация

KNN для задачи классификации:

1. Расчет расстояний: Для каждого нового примера (точки, которую нужно классифицировать) рассчитать расстояние до всех обучающих примеров.

2. Поиск K ближайших соседей: Отсортировать обучающие примеры по расстоянию до нового примера и выбрать K ближайших примеров. Это — ближайшие соседи.

3. Голосование: Определить класс, к которому относится новый пример, на основе голосования: класс, наиболее часто встречающийся среди K соседей, становится прогнозируемым классом для нового примера.

In [57]:
class KNNClassifier:
    def __init__(self, k=5):
        self.k = k

    def fit(self, X_train, y_train):
        self.X_train = X_train.to_numpy() if isinstance(X_train, pd.DataFrame) else X_train
        self.y_train = y_train.to_numpy() if isinstance(y_train, pd.Series) else y_train

    def predict(self, X_test):
        X_test = X_test.to_numpy() if isinstance(X_test, pd.DataFrame) else X_test
        return np.array([self._predict(x) for x in X_test])

    def _predict(self, x):
        x = np.array(x).reshape(1, -1)
        distances = distance.cdist(self.X_train, x, 'euclidean').flatten()
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0]

##### Регрессия

KNN для задачи регрессии:

1. Расчет расстояний: Аналогично классификации, рассчитать расстояние до всех обучающих примеров.

2. Поиск K ближайших соседей: Отсортировать обучающие примеры по расстоянию и выбрать K ближайших.

3. Среднее значение: Вместо голосования для задачи регрессии используется среднее значение целевых переменных K соседей.

In [58]:
class KNNRegressor:
    def __init__(self, k=5):
        self.k = k

    def fit(self, X_train, y_train):
        self.X_train = X_train.to_numpy() if isinstance(X_train, pd.DataFrame) else X_train
        self.y_train = y_train.to_numpy() if isinstance(y_train, pd.Series) else y_train

    def predict(self, X_test):
        X_test = X_test.to_numpy() if isinstance(X_test, pd.DataFrame) else X_test
        return np.array([self._predict(x) for x in X_test])

    def _predict(self, x):
        distances = [np.linalg.norm(x - x_train) for x_train in self.X_train]
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        return np.mean(k_nearest_labels)

### Обучение имплементированной модели

##### Классификация

Подготовка данных:

In [59]:
X_train_rice, X_test_rice, y_train_rice, y_test_rice = train_test_split(X_rice, y_rice, test_size=0.2, random_state=42)

imputer = SimpleImputer(strategy='mean')
X_train_rice = pd.DataFrame(imputer.fit_transform(X_train_rice))
X_test_rice = pd.DataFrame(imputer.transform(X_test_rice))

knn_classifier = KNNClassifier(k=5)

Обучение модели:

In [60]:
knn_classifier.fit(X_train_rice, y_train_rice)

##### Регрессия

Подготовка данных:

In [61]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)

imputer = SimpleImputer(strategy='mean')
X_train_gold = imputer.fit_transform(X_train_gold)
X_test_gold = imputer.transform(X_test_gold)

mask = ~np.isnan(y_train_gold)
X_train_gold = X_train_gold[mask]
y_train_gold = y_train_gold[mask]
mask_test = ~np.isnan(y_test_gold)
X_test_gold = X_test_gold[mask_test]
y_test_gold = y_test_gold[mask_test]

knn_regressor = KNNRegressor(k=5)

Обучение модели:

In [62]:
knn_regressor.fit(X_train_gold, y_train_gold)

### Оценка качества имплементированных моделей

##### Классификация

Оценка качества модели:

In [63]:
y_pred_rice = knn_classifier.predict(X_test_rice)

In [64]:
accuracy = accuracy_score(y_test_rice, y_pred_rice)
f1 = f1_score(y_test_rice, y_pred_rice, average='weighted')
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Accuracy: 0.9234
F1 Score: 0.9229124574106105


##### Регрессия

Оценка качества модели:

In [65]:
y_pred_gold = knn_regressor.predict(X_test_gold)

In [66]:
mae = mean_absolute_error(y_test_gold, y_pred_gold)
r2 = r2_score(y_test_gold, y_pred_gold)
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")

Mean Absolute Error: 3605268.701468624
R-squared: 0.26023132479378674


### Сравнение результатов первоначального бейзлайна и имплементированных моделей

##### Классификация

Видим, что результаты имплементированной модели немного лучше:

In [67]:
print(f"Accuracy difference: {accuracy-accuracy_origin}")
print(f"F1 Score difference: {f1-f1_origin}")

Accuracy difference: 0.0003333333333332966
F1 Score difference: 0.00039601430798941184


##### Регрессия

Видим, что результаты идентичны:

In [68]:
print(f"Mean Absolute Error difference: {mae_origin-mae}")
print(f"R-squared difference: {r2-r2_origin}")

Mean Absolute Error difference: 0.0
R-squared difference: 0.0


### Выводы

Оценка качества нашей имплементированной регрессионной модели оказалась идентичной встроенной регресионной модели.
Оценка качества нашей имплементированной модели для классификации оказалась иной по сравнению со встроенной моделью. Скорей всего из-за того, что в нашей имплементации используется другой алгоритм метрики расстояния.

### Добавление техник из улучшенного бейзлайна

##### Классификация

Формирование улучшенного бейзлайна:

In [69]:
X_train_rice, X_test_rice, y_train_rice, y_test_rice = train_test_split(X_rice, y_rice, test_size=0.2, random_state=42)

imputer = SimpleImputer(strategy='mean')
X_train_rice = pd.DataFrame(imputer.fit_transform(X_train_rice))
X_test_rice = pd.DataFrame(imputer.transform(X_test_rice))

scaler_rice = StandardScaler()
X_train_rice = scaler_rice.fit_transform(X_train_rice)
X_test_rice = scaler_rice.transform(X_test_rice)
                                   
knn_classifier = KNNClassifier(k=3)

##### Регрессия

Немного изменим наш класс, добавив в него возможность выбора метрики расстояния и параметра weights:

In [70]:
class KNNRegressor:
    def __init__(self, k=5, metric='euclidean', weights='uniform'):
        self.k = k
        self.metric = metric
        self.weights = weights

    def fit(self, X_train, y_train):
        self.X_train = X_train.to_numpy() if isinstance(X_train, pd.DataFrame) else X_train
        self.y_train = y_train.to_numpy() if isinstance(y_train, pd.Series) else y_train

    def predict(self, X_test):
        X_test = X_test.to_numpy() if isinstance(X_test, pd.DataFrame) else X_test
        return np.array([self._predict(x) for x in X_test])

    def _predict(self, x):
        x = np.array(x).reshape(1, -1)
        distances = distance.cdist(self.X_train, x, metric=self.metric).flatten()
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_distances = distances[k_indices]
        k_nearest_labels = [self.y_train[i] for i in k_indices]

        if self.weights == 'uniform':
            return np.mean(k_nearest_labels)
        elif self.weights == 'distance':
            # Избегаем деления на ноль, если расстояние равно нулю
            weights = 1 / (k_nearest_distances + 1e-10)
            return np.dot(weights, k_nearest_labels) / np.sum(weights)
        else:
            raise ValueError("Параметр weights должен быть 'uniform' или 'distance'.")

Формирование улучшенного бейзлайна:

In [71]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)

imputer = SimpleImputer(strategy='mean')
X_train_gold = imputer.fit_transform(X_train_gold)
X_test_gold = imputer.transform(X_test_gold)

mask = ~np.isnan(y_train_gold)
X_train_gold = X_train_gold[mask]
y_train_gold = y_train_gold[mask]
mask_test = ~np.isnan(y_test_gold)
X_test_gold = X_test_gold[mask_test]
y_test_gold = y_test_gold[mask_test]

scaler_gold = StandardScaler()
X_train_gold = scaler_gold.fit_transform(X_train_gold)
X_test_gold = scaler_gold.transform(X_test_gold)

knn_regressor = KNNRegressor(k=4, weights="distance", metric="cityblock")

### Обучение на улучшенном бейзлайне

##### Классификация

In [72]:
knn_classifier.fit(X_train_rice, y_train_rice)

##### Регрессия

In [73]:
knn_regressor.fit(X_train_gold, y_train_gold)

### Оценка качества моделей на улучшенном бейзлайне

##### Классификация

In [74]:
y_pred_rice = knn_classifier.predict(X_test_rice)
accuracy = accuracy_score(y_test_rice, y_pred_rice)
f1 = f1_score(y_test_rice, y_pred_rice, average='weighted')
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Accuracy: 0.9978666666666667
F1 Score: 0.9978677682961133


##### Регрессия

In [75]:
y_pred_gold = knn_regressor.predict(X_test_gold)
mae = mean_absolute_error(y_test_gold, y_pred_gold)
r2 = r2_score(y_test_gold, y_pred_gold)
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")

Mean Absolute Error: 2398788.4102571798
R-squared: 0.6048118747567031


### Сравнение результатов

##### Классификация

In [76]:
print(f"Accuracy difference: {accuracy-accuracy_improved}")
print(f"F1 Score difference: {f1-f1_improved}")

Accuracy difference: 0.0003333333333332966
F1 Score difference: 0.00033308026548106717


##### Регрессия

In [77]:
print(f"Mean Absolute Error difference: {mae-mae_improved}")
print(f"R-squared difference: {r2-r2_improved}")

Mean Absolute Error difference: -59727.67759328801
R-squared difference: 0.017842518554244124


### Выводы

Результаты получились следующие: качество наших имплементаций для задач классификации и регрессии оказалось немного выше улучшенного бейзлайна на готовой реализации.